In [16]:
import pandas as pd
from glob import glob
import os

# read all files in the folder

current_folder = 'R-PV-Forecasting/dataset'
all_files = glob(current_folder + "/*.xls")

# load dataset from folder dataset which are xls files
if all_files:
	dataset = pd.concat([pd.read_excel(file, header=3) for file in all_files])
	# show header title
	print(dataset.columns)
else:
	print("No files found in the folder.")



Index(['DateTime', 'Most recent forecast [MW]', 'Day-Ahead forecast [MW]',
       'Week-Ahead forecast [MW]', 'Real-time Upscaled Measurement [MW]',
       'Corrected Upscaled Measurement [MW]', 'Monitored Capacity [MWp]',
       'Day-Ahead forecast (11h00) [MW]'],
      dtype='object')


In [17]:
# shwo the first 5 rows of the dataset
print(dataset.head())


           DateTime  Most recent forecast [MW]  Day-Ahead forecast [MW]  \
0  01/02/2012 00:00                        0.0                      0.0   
1  01/02/2012 00:15                        0.0                      0.0   
2  01/02/2012 00:30                        0.0                      0.0   
3  01/02/2012 00:45                        0.0                      0.0   
4  01/02/2012 01:00                        0.0                      0.0   

   Week-Ahead forecast [MW]  Real-time Upscaled Measurement [MW]  \
0                       0.0                                  0.0   
1                       0.0                                  0.0   
2                       0.0                                  0.0   
3                       0.0                                  0.0   
4                       0.0                                  0.0   

   Corrected Upscaled Measurement [MW]  Monitored Capacity [MWp]  \
0                                  0.0                    400.02   
1   

In [18]:
# Delete the first 3 columns
dataset = dataset.drop(dataset.columns[[0, 1, 2]], axis=1)

# show the first 5 rows of the dataset
print(dataset.head())

   Week-Ahead forecast [MW]  Real-time Upscaled Measurement [MW]  \
0                       0.0                                  0.0   
1                       0.0                                  0.0   
2                       0.0                                  0.0   
3                       0.0                                  0.0   
4                       0.0                                  0.0   

   Corrected Upscaled Measurement [MW]  Monitored Capacity [MWp]  \
0                                  0.0                    400.02   
1                                  0.0                    400.02   
2                                  0.0                    400.02   
3                                  0.0                    400.02   
4                                  0.0                    400.02   

   Day-Ahead forecast (11h00) [MW]  
0                              0.0  
1                              0.0  
2                              0.0  
3                              0.0

In [19]:
import pandas as pd
import os

# Initialize an empty list to store dataframes
dataframes = []
save_path = os.path.join(os.getcwd(), 'dataset_merge')

power_columns = ["Most_Recent_Forecast_MW", "Day_Ahead_Forecast_MW", "Week_Ahead_Forecast_MW", 
                      "Real_Time_Measurement_MW", "Corrected_Measurement_MW", "Monitored_Capacity_MWp", "Day_Ahead_11h00_MW"]

# Iterate over each file in the dataset folder
for file in os.listdir(current_folder):
    if file.endswith('.xls'):
        file_path = os.path.join(current_folder, file)
        
        # Load the Excel file
        df = pd.read_excel(file_path)
        
        # Rename columns for easier access
        df.columns = ["DateTime", "Most_Recent_Forecast_MW", "Day_Ahead_Forecast_MW", "Week_Ahead_Forecast_MW", 
                      "Real_Time_Measurement_MW", "Corrected_Measurement_MW", "Monitored_Capacity_MWp", "Day_Ahead_11h00_MW"]
        
        # Drop the first three rows which contain metadata
        df = df.iloc[3:].reset_index(drop=True)
        
        # Convert DateTime column to actual datetime format
        df["DateTime"] = pd.to_datetime(df["DateTime"], format="%d/%m/%Y %H:%M", errors='coerce')
        
        # Convert power columns to numeric values
        df[power_columns] = df[power_columns].apply(pd.to_numeric, errors="coerce")
        
        # Append the cleaned dataframe to the list
        dataframes.append(df)

# Concatenate all dataframes into a single dataframe
merged_df = pd.concat(dataframes, ignore_index=True)

# Ensure the save path directory exists
os.makedirs(save_path, exist_ok=True)

# Save the merged dataframe to a new Excel file
merged_df.to_excel(os.path.join(save_path, 'dataset_merge.xlsx'), index=False)
# save to pandas object
merged_df.to_pickle(os.path.join(save_path, 'dataset_merge.pkl'))

# Display the merged dataframe
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Merged Solar Power Output Data", dataframe=merged_df)

Merged Solar Power Output Data


In [20]:
# cheack for missing values
missing_values = merged_df.isnull().sum()
print(f'Missing values in the merged dataset: \n{missing_values}')

Missing values in the merged dataset: 
DateTime                         0
Most_Recent_Forecast_MW          0
Day_Ahead_Forecast_MW       109728
Week_Ahead_Forecast_MW      159655
Real_Time_Measurement_MW         0
Corrected_Measurement_MW         0
Monitored_Capacity_MWp           0
Day_Ahead_11h00_MW            1152
dtype: int64


In [22]:
# find minimum and maximum Real_Time_Measurement_MW  just 2012
merged_df_2012 = merged_df[merged_df["DateTime"].dt.year == 2012]
min_power_2012 = merged_df_2012["Real_Time_Measurement_MW"].min()
max_power_2012 = merged_df_2012["Real_Time_Measurement_MW"].max()
print(f'Minimum power output in 2012: {min_power_2012} MW')
print(f'Maximum power output in 2012: {max_power_2012} MW')


# find average power output in 2012
average_power_2012 = merged_df_2012["Real_Time_Measurement_MW"].mean()
print(f'Average power output in 2012: {average_power_2012} MW')

# find the total power output in 2012
total_power_2012 = merged_df_2012["Real_Time_Measurement_MW"].sum()
print(f'Total power output in 2012: {total_power_2012} MW')


Minimum power output in 2012: 0.0 MW
Maximum power output in 2012: 187.98 MW
Average power output in 2012: 1.4247901119402986 MW
Total power output in 2012: 45821.25 MW


In [29]:
import pandas as pd

# Ensure 'DateTime' column is in datetime format
merged_df["DateTime"] = pd.to_datetime(merged_df["DateTime"], errors="coerce")

# Define the range of years (2012 to 2024)
years = list(range(2012, 2025))

# Create an empty list to store yearly results
yearly_stats = []

# Loop through each year and calculate min, max, avg, and total
for year in years:
    df_year = merged_df[merged_df["DateTime"].dt.year == year]
    
    if not df_year.empty and "Real_Time_Measurement_MW" in df_year.columns:
        yearly_stats.append({
            "Year": year,
            "Minimum Power (MW)": df_year["Real_Time_Measurement_MW"].min(),
            "Maximum Power (MW)": df_year["Real_Time_Measurement_MW"].max(),
            "Average Power (MW)": df_year["Real_Time_Measurement_MW"].mean(),
            "Total Power (MW)": df_year["Real_Time_Measurement_MW"].sum(),
        })
    else:
        print(f"No data available for {year} or column missing.")

# Convert results to a DataFrame
yearly_stats_df = pd.DataFrame(yearly_stats)

# Display the results in an interactive table
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Yearly_Power_Stats", dataframe=yearly_stats_df)

# Optionally, save results to an Excel file
yearly_stats_df.to_excel("Yearly_Power_Stats.xlsx", index=False)
print("Yearly power statistics saved to Yearly_Power_Stats.xlsx")


Yearly_Power_Stats


Yearly power statistics saved to Yearly_Power_Stats.xlsx


In [30]:
# export the dataset to a new Excel file named values.xlsx
merged_df.to_excel("values.xlsx", index=False)
print("Dataset saved to values.xlsx")



Dataset saved to values.xlsx
